In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GenerationConfig, TrainingArguments, Trainer

import torch
import time

import evaluate

import numpy as np
import pandas as pd

In [30]:
huggingface_dataset_name = "squad"
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [31]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print(f"Example {i + 1}")

    print(dash_line)
    print(f"context: \n{dataset['train'][index]['context']}")
    print(f"question: \n{dataset['train'][index]['question']}")
    print(f"answers: \n{dataset['train'][index]['answers']['text'][0]}")


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
context: 
Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College of

In [32]:
model_name = "google-t5/t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 零样本微调

In [33]:
for i, index in enumerate(example_indices):
    context = dataset['train'][index]['context']
    question = dataset['train'][index]['question']
    answer = dataset['train'][index]['answers']['text'][0]

    prompt = f"Given the following context and an certain answer, please generate a certain question from the context and answer:\n \Context: {context} \n \Answer: {answer}.\n Question:"

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = tokenizer.decode(original_model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True)

    print(dash_line)
    print(f"Example {i+1}")
    print(f"context: \n{context}")
    print(f"question: \n{question}")
    print(f"answer: \n{answer}")
    print(f"Generated question: \n{outputs}")

---------------------------------------------------------------------------------------------------
Example 1
context: 
Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College of the First Year of Studies, 25% have indicated they plan to study in the liberal arts or social scie

### 少样本微调

In [34]:
def make_prompt(example_indices_full, example_index_to_questions):
    prompt = ""

    for index in example_indices_full:
        context = dataset['train'][index]['context']
        question = dataset['train'][index]['question']
        answer = dataset['train'][index]['answers']['text'][0]
        prompt += f"Given the following context and an certain answer, please generate a certain question from the context and answer:\n Context: {context} \n Answer: {answer} \n Question: {question}\n\n"
    context = dataset['train'][example_index_to_questions]['context']
    question = dataset['train'][example_index_to_questions]['question']
    answer = dataset['train'][example_index_to_questions]['answers']['text'][0]
    prompt += f"Given the following context and an certain answer, please generate a certain question from the context and answer:\n Context: {context} \n Answer: {answer}.\n Question:"
    return prompt

example_indices_full = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
example_index_to_questions = 10
few_shot_prompt = make_prompt(example_indices_full, example_index_to_questions)
print(few_shot_prompt)

Given the following context and an certain answer, please generate a certain question from the context and answer:
 Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. 
 Answer: Saint Bernadette Soubirous 
 Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?

Given the following context and an certain answer, please

In [35]:
context = dataset['train'][example_index_to_questions]['context']
question = dataset['train'][example_index_to_questions]['question']
answer = dataset['train'][example_index_to_questions]['answers']['text'][0]

inputs = tokenizer(few_shot_prompt, return_tensors="pt")
outputs = tokenizer.decode(original_model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True)

print(dash_line)
print(f"context: \n{context}")
print(f"question: \n{question}")
print(f"answer: \n{answer}")
print(f"Generated question: \n{outputs}")

---------------------------------------------------------------------------------------------------
context: 
The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.
question: 
Where is the headquarters of the Congregation of the Holy Cross?
answer: 
Rome
Generated question: 
True
